# EWS and ML classifier evalutaion on COVID respiratory deterioration from EHR

Code to evaluate various EWS and ML classifiers on predicting respiratory deterioration (surrogated by escalation in repiratory care apparatus/ICU admission) in COVID patients, given static EHR readings for the patient.

Logic:

- Nested 5-fold CV on model and all data over hyperparameters for model. Scored via 'roc_auc'.

In [ ]:
from pathlib import Path
from tqdm.auto import tqdm
from cycler import cycler

import pandas as pd
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt; plt.ioff()

import sklearn.metrics as metrics
from sklearn.svm import SVR
from sklearn.metrics import confusion_matrix, get_scorer, make_scorer, recall_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, BayesianRidge
from sklearn.experimental import enable_iterative_imputer; from sklearn.impute import IterativeImputer, KNNImputer
from sklearn.preprocessing import scale
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_validate, GridSearchCV, KFold, StratifiedKFold, RepeatedStratifiedKFold
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel

from imblearn.ensemble import BalancedRandomForestClassifier, BalancedBaggingClassifier
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import RandomOverSampler

from autoimpute.imputations import SingleImputer, MultipleImputer

import ews_thresholds as ews

%reload_ext autoreload
%autoreload 2

In [ ]:
def mean_confidence_interval(a, sd=None, n=None, confidence=0.95, verbose=True):
    """
    Given a list of values, returns a string of form: 
    'mean (confidence, interval)'
    """
    if n is None:
        n = len(a)
        mean = np.mean(a)
        se = scipy.stats.sem(a)
        h = se * scipy.stats.t.ppf((1 + confidence) / 2, n-1)
        left = mean - h
        right = mean + h
        
    else:
        mean = a
        left, right = scipy.stats.norm.interval(alpha=confidence, loc=mean, scale=sd/np.sqrt(n))
    
    if verbose:
        return f'{mean:.2f} ({left:.2f}-{right:.2f})'
    else:
        return mean, (mean - h, mean + h)

In [ ]:
def reliability_measures(pred, actual):
    """
    Given a set or predicted values and their true values, returns
    the accuracy, sensitivity, specificity, and precision.
    
    Given the measures are binary and symmetric, the order of pred/actual
    can be reversed with no change in behaviour.
    """
    tn, fp, fn, tp = confusion_matrix(pred, actual).ravel()
    acc = (tn+tp) / (tn+fp+fn+tp)
    sen = tp / (tp+fn)
    sps = tn / (tn+fp)
    prs = tp / (tp+fp)
    
    return acc, sen, sps, prs

In [ ]:
def data_imputing(data, method='median', baseline_method='mean'):
    """
    Data imputation function.
    """
    estimator_dict = {'RF': RandomForestRegressor(max_depth=2, random_state=random_state),
                      'SVR': SVR(C=1.0, epsilon=0.3),
                      'GBT': GradientBoostingRegressor(n_estimators=10, random_state=random_state),
                      'GPR': GaussianProcessRegressor(kernel=RBF(0.1) + WhiteKernel(0.01), n_restarts_optimizer=9),
                      'BayesianRidge': BayesianRidge()}
    
    if method in ('mean', 'mode', 'median', 'norm'):
        imputer = SingleImputer(strategy=method)
    elif method in ('least squares', 'stochastic', 'multinomial logistic'):
        imputer = MultipleImputer(1, strategy=method, return_list=True)
    elif method == 'KNN':
        imputer = KNNImputer(n_neighbors=2, weights="uniform")
    elif method in ('BayesianRidge', 'RF', 'GBT', 'SVR', 'GPR'):
        imputer = IterativeImputer(random_state=random_state, estimator=estimator_dict[method])
    
    # Imputing baseline values with population means
    if baseline_method:
        bl_imputer = SingleImputer(strategy=baseline_method)
        bl_cols = [col for col in data.columns if 'Baseline' in col]
        data[bl_cols] = bl_imputer.fit_transform(data[bl_cols])
    
    # Imputing values with method
    data = imputer.fit_transform(data)
    
    if method in ('least squares', 'stochastic', 'multinomial logistic'):
        data = data[0][1]
        
    return data

In [ ]:
def data_manipulation(data, imputation_method='median', include_fio2=False):
    """
    Data imputation and unit adjustment.
    """
    # Imputing numeric columns
    if not include_fio2:
        data.drop(['Blood_Gas FIO2'], axis=1, inplace=True)
        # [col for col in data.columns if 'fio2' in col.lower()]
    data.loc[:,"Vital_Signs masktyp"].fillna(value=0, inplace=True) #important point as our prediction is based on the masktype (level of care)
    gen = data[["hadm_id", "sex"]]
    data.drop("sex", axis=1, inplace=True) # Drop gender while imputing
    data = data_imputing(data, imputation_method)
    data = pd.merge(data, gen, on="hadm_id") # Re-add gender
    data = data.drop_duplicates().reset_index().drop("index", axis=1)
    
    data['sex'] = data['sex'].replace({"M":0, "F":1})
    data.loc[data['Vital_Signs avpu'] == -1, 'Vital_Signs avpu'] = 0

    # for the blood_test_gas and all_features_etc combined feature sets
    
    return data

In [ ]:
def filter_feature(df, feature_type, top_n_features=False):
    """
    Given complete dataset, outputs subset of features (with label col).
    """
    
    if top_n_features:
        return df[top_n_features + ['label']].copy()
    
    # Grabbing feature-sets' columns
    EWS_bloods = ['Blood_Test Albumin-g/L','Blood_Test Creatinine-umol/L', 'Blood_Test Haemoglobin-g/dL',
                  'Blood_Test Urea-mmol/L', 'Blood_Test WhiteCells-x10^9/L',
                  # Does it matter which is used?
                  'Blood_Test Potassium-mmol/L', 'Blood_Test Sodium-mmol/L']
    CEWS_vitals = ['Vital_Signs HR', 'Vital_Signs RR', 'Vital_Signs SBP', 'Vital_Signs SPO2', 'Vital_Signs TEMP']
    NEWS_vitals = CEWS_vitals + ['Vital_Signs avpu', 'Vital_Signs masktyp']
    
    EWS_misc = ['age', 'sex', 'hrs_from_adm_lab']
    
    vital_baseline_delta = [c for c in df.columns if 'Vital_Signs Baseline' in c]
    baseline_delta = [c for c in df.columns if 'Baseline' in c]
    variations = [c for c in df.columns if ('Delta_Mean' in c or 'Var_Mean' in c or 'Max_Min' in c) and c not in baseline_delta]
    
    vitals = [c for c in df.columns if 'Vital_Signs' in c and c not in baseline_delta and c not in variations]
    bloodgas = [c for c in df.columns if 'Blood_Gas' in c and c not in baseline_delta and c not in variations]
    bloodtest = [c for c in df.columns if 'Blood_Test' in c and c not in baseline_delta and c not in variations]
    allfeats = vitals + bloodtest + bloodgas + EWS_misc
    
    aews_feats = NEWS_vitals + ['age']
    ldtews_feats = EWS_bloods + ['sex']
    ldtews_news_feats = NEWS_vitals + ldtews_feats + ['hrs_from_adm_lab']
    
    
    feature_cols_dict = {'Vitals': vitals,
                         'Blood_Test': bloodtest,
                         'Blood_Gas': bloodgas,
                         
                         'Vitals&Vars': vitals + variations,
                         'Vitals&Vars&Baseline_Delta': vitals + variations + vital_baseline_delta,
                         
                         'All Features': allfeats,
                         'All Features&Vars': allfeats + variations,
                         'All Features&Vars&Baseline_Delta': allfeats + variations + baseline_delta,
                         
                         'EWS_cols': NEWS_vitals + EWS_bloods + EWS_misc,
                         'EWS_Bloods': EWS_bloods,
                         'NEWS_feats': NEWS_vitals,
                         'CEWS_feats': CEWS_vitals,
                         'MCEWS_feats': NEWS_vitals,
                         'AEWS_feats': aews_feats,
                         'LDTEWS_feats': ldtews_feats,
                         'LDTEWS_NEWS_feats': ldtews_news_feats}
    
    filtered_df = df[feature_cols_dict[feature_type] + ['label'].copy()
    
    return filtered_df

In [ ]:
def misclass_stats(test_data, pred_acc):
    """
    Saves two files - one wth the ID's of patients correctly classified on last run of test-data 
    of last model/feature set tested;
    and one with that of those incorrectly classified.
    
    Ethnicity codes available at:
    https://peoplefirst.nhsbt.nhs.uk/People%20First%20-%20Document%20Library/Pay/General%20-%20Ethnic%20origin%20codes%202015.pdf
    
    white = ['A','B','C']
    unknowns = [99,'Z'] 
    bame = [code for code in codelist if code not in white and not in unknowns]
    
    # 99 is Unknown: https://www.datadictionary.nhs.uk/data_elements/ethnic_category.html
    # 'Z' is unstated: https://www.datadictionary.nhs.uk/attributes/ethnic_category_code_2001.html
    
    """

    testdf = test_data.copy()

    testdf["pred_lbls"] = pred_acc
    testdf["slicer"] = testdf.label != testdf.pred_lbls
    
    misclassified = testdf[testdf.slicer==1][["hadm_id","age","sex"]]
    correctly_classified = testdf[testdf.slicer==0][["hadm_id","age","sex"]]
    
    misclassified.to_csv(file_loc / 'results' / "misclassified_gen_age.csv")
    correctly_classified.to_csv(file_loc / 'results' / "correctly_classified_gen_age.csv")

In [ ]:
def plot_mean_auroc(tprs, aucs, method, f_type, ax=None, save_fig=True):
    """
    Given multiple ROC performances, plots mean performance.
    """
    
    mean_fpr = np.linspace(0, 1, 100)
    
    if ax==None:
        ax = plt.gca()
        
    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = metrics.auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    ax.plot(mean_fpr, mean_tpr,
            label=f'Mean ROC (AUC = {mean_auc:.2f} $\pm$ {std_auc:.2f}) {method}',
            lw=2, alpha=.8)

    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    
    if 'EWS' not in method or method=='LDTEWS_NEWS':
        ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2, label=r'$\pm$ 1 std. dev.')
        ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', label='Chance', alpha=.8)
    else:
        ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2, label=None)

    ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05], title="Receiver operating characteristic")
    ax.legend(loc="lower right")


In [ ]:
def nested_cv(X, y, model, n_repeats=40, oversample=True):
    """
    Given model type, and default hyperparameter list, performs nested CV (5x5) grid search.
    
    Outputs outer CV results for each fold on multiple metrics (acc, sen, sps, prs, auc etc).
    """
    
    # Define models & params
    t = {'n_estimators': [10, 20, 50, 100, 500, 1000]}
    
    model_params = {'LR': {'C': [1, 3, 5, 10, 100]}, 'MLP': {'hidden_layer_sizes': [10, 30, 50, 100, 200]}, 
                    'RF':t, 'GBT':t, 'BRF':t, 'BBag':t}
    
    model_clfs = {'LR': LogisticRegression(random_state=random_state),
                  'RF': RandomForestClassifier(random_state=random_state), 
                  'GBT': GradientBoostingClassifier(random_state=random_state), 
                  'BRF': BalancedRandomForestClassifier(random_state=random_state), 
                  'MLP': MLPClassifier(random_state=random_state),
                  'BBag': BalancedBaggingClassifier(random_state=random_state)}
    
    clf = model_clfs[model]
    
    # Grab "hypo"-parameters for nested CV
    param_grid = model_params[model]
    
    inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state) 
    
    # Create pipeline to do class-balancing in internal CV
    if oversample:
        outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
        oversampler = RandomOverSampler(sampling_strategy='minority', random_state=random_state)
        pipeline = make_pipeline(oversampler, clf)
        param_grid = {f'{type(clf).__name__.lower()}__{key}':value for key, value in param_grid.items()}
    else:
        outer_cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=n_repeats, shuffle=True, random_state=random_state)
        pipeline = clf
        
    # Scaling data to encourage alg convergence
    if method=='LR':
        X = scale(X)
    
    # Define metrics for scoring
    scoring = {'AUC': 'roc_auc', 
               'Accuracy': get_scorer('accuracy'),
               'Precision': get_scorer('precision'),
               'Sensitivity': get_scorer('recall'),
               'Specificity': make_scorer(recall_score, pos_label=0),
               'Balanced_accuracy': get_scorer('balanced_accuracy')}
    
    # Non_nested parameter search and scoring
    clf = GridSearchCV(pipeline, param_grid, cv=inner_cv, scoring=scoring, refit='AUC')
    clf.fit(X, y)
    print('Inner CV completed.')
    
    # Nested results
    nested_scores = cross_validate(clf, X, y, cv=outer_cv, scoring=scoring, return_estimator=True)
    print('Outer CV completed.')
        
    return nested_scores

In [ ]:
def classification_test(Xtrain, ytrain, Xtest, ytest, model, EWS_threshold):
    """
    Grabs appropriate classifier based on model name and applies to train data.
    """
    
    score_test = ews.calculate_ews(Xtest, model)
    prob_test = score_test[f'{model}_prob']
    
    fpr, tpr, _ = metrics.roc_curve(ytest, prob_test)
    roc_auc_test = metrics.auc(fpr, tpr)
    test = {'sen':[], 'sps':[], 'acc':[], 'prs':[], 'roc_auc':roc_auc_test, 'thresh':[]}
    
    if EWS_threshold == 'default':
        
        default_thresh = score_thresh_dict[model]['default']
        pred_test = score_test[model]>=default_thresh

        acc, sen, sps, prs = reliability_measures(ytest, pred_test)
        
        test['acc'].append(acc)
        test['sen'].append(sen)
        test['sps'].append(sps)
        test['prs'].append(prs)
        test['thresh'].append(default_thresh)
        
        test['fpr'] = fpr
        test['tpr'] = tpr
        
        return test

    # Optimising thresholds
    elif EWS_threshold == 'optimise':
        
        # Note: not (necessarily) a probability - the value output by the EWS (e.g. between 0 and 20)
        score_train = ews.calculate_ews(Xtrain, model)
        prob_train = score_train[f'{model}_prob']
        
        # ROC_AUC doesn't vary with threshold
        fpr, tpr, _ = metrics.roc_curve(ytrain, prob_train)
        roc_auc_train = metrics.auc(fpr, tpr)

        # Containers to hold lists of values for each train/test run for each EWS threshold
        train = {'sen':[], 'sps':[], 'bal_acc':[], 'acc':[], 'prs':[], 'roc_auc':roc_auc_train, 'thresh':[]}
        
        # Calculate metrics for each threshold
        for thresh in score_thresh_dict[model]['range']:
            
            # On train
            pred_train = score_train[model]>=thresh
            acc, sen, sps, prs = reliability_measures(ytrain,pred_train)
            bal_acc = (sen+sps)/2
            
            train['bal_acc'].append(bal_acc)
            train['acc'].append(acc)
            train['sen'].append(sen)
            train['sps'].append(sps)
            train['prs'].append(prs)
            train['thresh'].append(thresh)
            
            # On test
            pred_test = score_test[model]>=thresh
            acc, sen, sps, prs = reliability_measures(ytest,pred_test)
            
            test['acc'].append(acc)
            test['sen'].append(sen)
            test['sps'].append(sps)
            test['prs'].append(prs)
            test['thresh'].append(thresh)

        return train, test

In [ ]:
def main(raw_data,feature_types,methods,EWS_threshold='default',plot_curves=False,split_ids=False,top_n_features=None):
    """
    Main program. For each:
    
    - Feature set
    - ML method
    
    1. Takes the input data
    2. Appropriately subsets its columns
    3. Performes (Nested) GridSearchCV (data-balancing / hyper-param tuning / model training)
    7. Outputs avg performance on test sets across folds in CV.
    
    If the method is an EWS system, depending on EWS_threshold value will either:
    
    - Output performance of EWS on above CV's test sets.
    - Output performance of EWS with (balanced) accuracy-optimised threshold (as per GridSearchCV logic) on above.
    """
    
    results_df = pd.DataFrame({"feature_type":[], "method":[], "acc":[], "sen":[],"sps":[], "prs":[], "AUC":[]})

    for f_type in tqdm(feature_types, desc='Feature set'):
        data = filter_feature(raw_data, f_type, top_n_features)
        if plot_curves:
            fig, ax = plt.subplots()
        
        for method in tqdm(methods, desc='Model', position=1, leave=bool(f_type==feature_types[-1])): 
            n_splits=5
            skf = StratifiedKFold(n_splits=n_splits, random_state=random_state, shuffle=True)
            
            ####################################
            # Perform Nested GridSearchCV for ML models
            ####################################
            if 'EWS' not in method:
                X = data.drop('label', axis=1).values
                y = data['label'].values.ravel()
                
                nested_scores = nested_cv(X, y, method)
                
                accs = nested_scores['test_Accuracy']
                sens = nested_scores['test_Sensitivity']
                spss = nested_scores['test_Specificity']
                prss = nested_scores['test_Precision']
                aucs = nested_scores['test_AUC']
                
                results_df = results_df.append({"feature_type":f_type,"method":method,
                                                "acc":mean_confidence_interval(accs),
                                                "sen":mean_confidence_interval(sens),
                                                "sps":mean_confidence_interval(spss),
                                                "prs":mean_confidence_interval(prss),
                                                "AUC":mean_confidence_interval(aucs)},
                                               ignore_index=True)
                
                # Re-iterating through test-data splits to calculate fpr/tpr for plotting ROC curves
                if plot_curves:
                    tprs = []
                    for i, (train_index, test_index) in enumerate(skf.split(data, data['label'])):
                        X_test = X[test_index]
                        y_test = y[test_index]
                        
                        clf_fold = nested_scores['estimator'][i]

                        viz = metrics.plot_roc_curve(clf_fold, X_test, y_test, name=f'ROC fold {i}',
                                                     alpha=0.3, lw=1, ax=ax)
                        
                        mean_fpr = np.linspace(0, 1, 100)
                        interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
                        interp_tpr[0] = 0.0
                        tprs.append(interp_tpr)

            ####################################
            # Perform CV for EWS 
            # (using same random seed to ensure comparative testing with ML results)
            ####################################
            else:
                data = filter_feature(raw_data, f'{method}_feats')
                # Output of metrics on train/test for optimising threshes
                train_folds = []
                test_folds = []
                
                # Need to manually go through each fold for EWS since they are not defined as classifiers
                for i, (train_index, test_index) in tqdm(enumerate(skf.split(data, data['label'])), desc=f'Stratified {n_splits}-fold', total=n_splits, leave=False):
                    train_data, test_data = data.loc[train_index,:], data.loc[test_index,:]

                    # Split labels and features
                    ytrain = train_data[['label']].values
                    ytest = test_data[['label']].values
                    
                    # Keep column names for EWS calculations
                    Xtrain = train_data.drop('label', axis=1)
                    Xtest = test_data.drop('label', axis=1)
                    
                    # Calculating performance metrics
                    if EWS_threshold == 'default':
                        test = classification_test(Xtrain,ytrain,Xtest,ytest,method,EWS_threshold)
                        
                        if plot_curves:
                            # Plotting fold's ROC
                            viz = metrics.RocCurveDisplay(fpr=test['fpr'], tpr=test['tpr'], roc_auc=test['roc_auc'], 
                                                          estimator_name=f'ROC fold {i}')#,
                                                          #alpha=0.3, lw=1, ax=ax)

                            mean_fpr = np.linspace(0, 1, 100)
                            interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
                            interp_tpr[0] = 0.0
                            test['interp_tpr'] = interp_tpr
                        
                        test_folds.append(test)
    
                    elif EWS_threshold == 'optimise':
                        train, test = classification_test(Xtrain,ytrain,Xtest,ytest,method,EWS_threshold)
                        train_folds.append(train)
                        test_folds.append(test)

                if EWS_threshold == 'default':
                    i=0
                    
                elif EWS_threshold == 'optimise':
                    # Get mean performance of each threshold across train folds
                    mean_acc = [np.mean([fold['bal_acc'][j] for fold in train_folds]) for j in range(len(train_folds[0]['acc']))]
                    
                    # Grab best performing thresh on train
                    i = np.argmax(mean_acc)
                
                # Optimised/default threshold
                thresh = test_folds[0]['thresh'][i]
                    
                # Optimised/default metrics over all folds
                acc = [fold['acc'][i] for fold in test_folds]
                sen = [fold['sen'][i] for fold in test_folds]
                sps = [fold['sps'][i] for fold in test_folds]
                prs = [fold['prs'][i] for fold in test_folds]
                roc_auc = [fold['roc_auc'] for fold in test_folds]
                
                # Add results to output
                results_df = results_df.append({"feature_type":f_type,
                                                "method":method,
                                                "threshold":thresh,
                                                "acc":mean_confidence_interval(acc), 
                                                "sen":mean_confidence_interval(sen), 
                                                "sps":mean_confidence_interval(sps), 
                                                "prs":mean_confidence_interval(prs), 
                                                "AUC":mean_confidence_interval(roc_auc)},
                                               ignore_index=True)
                    
                
            # Plot mean AUROC
            if plot_curves:
                if 'EWS' in method:
                    tprs = [fold['interp_tpr'] for fold in test_folds]
                    aucs = [fold['roc_auc'] for fold in test_folds]
                    
                plot_mean_auroc(tprs, aucs, method, f_type, ax=ax)
                
            feature_names = [col for col in data.columns.tolist() if col != 'label']
            #fimp_df = pd.DataFrame({"feature":feature_names, "feature_weight":importance})
            #fimp_df.sort_values("feature_weight",ascending=False,inplace=True)
                
    if False:
        misclass_stats(test_data, results['pred'])
    
    if plot_curves:
        plt.show()
        if 'EWS' in method:
            plt.savefig(file_loc / 'imgs' / 'EWS_roc.png')
        else:
            plt.savefig(file_loc / 'imgs' / '{method}_{f_type}.png')
    
    return results_df #, fimp_df

## Performance analysis

* Step 0a: Assess performance of EWS systems (default thresholds)
* Step 0b: Assess performance of EWS systems (thresholds optimised for balanced-accuracy on 5-fold CV)
* Step 1: Compare multiple ML models on EWS feature-sets to determine best performer.
* Step 2: Train and evaluate best performing model on extended feature sets.
* Step 3: ~Plot AUROC curve.~ TO IMPLEMENT AVG ACROSS CV
* Step 4: ~Display most important features.~ TO IMPLEMENT AVG ACROSS CV
* Step 5: Additional feature sets / data set evaluations (age, different lookack windows etc).

In [ ]:
# Global seed for reproducibility
random_state = 4

In [ ]:
# Loading raw data
file_loc = Path('Z:/netshares/ibme_chi/Projects_1/orchid/processing/SK/JA')
data = pd.read_csv(file_loc / 'data' / 'dataevents_updated2_N24_W24.csv')
data = data_manipulation(data)

### 0a: Assess performance of EWS systems (default thresholds)

In [ ]:
# Threshold ranges for each model
score_thresh_dict = {"NEWS": {'default':5, 'range':np.arange(0,21)},
                     "MCEWS": {'default':4, 'range':np.arange(0,21)}, 
                     "CEWS": {'default':4, 'range':np.arange(0,18)}, 
                     "AEWS": {'default':5, 'range':np.arange(0,20)}, 
                     "LDTEWS": {'default':0.33, 'range':np.arange(0,1,0.01)},
                     "LDTEWS_NEWS": {'default':0.27, 'range':np.arange(0,1,0.01)}}

# EWS models and set of columns for calculations
ews_methods = score_thresh_dict.keys()
ews_feature_types = ['EWS_cols']

In [ ]:
ews_df = main(data, ews_feature_types, ews_methods, plot_curves=True)
ews_df
# use df.to_latex(index=False) for exporting of results for paper

### 0b: Assess performance of EWS systems (thresholds optimised for balanced-accuracy on 5-fold CV)

In [ ]:
opt_ews_df = main(data, ews_feature_types, ews_methods, EWS_threshold='optimise')

### 1: Compare multiple ML models on EWS feature-sets to determine best performer.

In [ ]:
####################
# imputation_methods
####################
imputation_methods = ['BayesianRidge', 'stochastic', 'mean', 'median'] 

##############
# Feature sets
##############
# For comparison with EWS
feature_types_ews = ['CEWS_feats', 'NEWS_feats', 'AEWS_feats', 'LDTEWS_feats', 'LDTEWS_NEWS_feats']
#feature_types_simple = ['Vitals', 'EWS_Bloods', 'Vital_Signs&EWS_Bloods']

# F1-F8
#feature_types = feature_types_simple + ['Blood_Test', 'Blood_Gas', 'Blood_Test_Gas', 'Vitals&Vars', 'All Features']
feature_types = ['Vitals', 'Blood_Test', 'Blood_Gas', 'All Readings']

# F9-F11
feature_types_extended = ['Vitals&Vars', 'All Features&Vars', 'Vitals&Vars&Baseline_Delta', 'All Features&Vars&Baseline_Delta']

###########
# ML models
###########
ml_methods = ['LR', 'BRF', 'GBT']

In [ ]:
results_df = main(data, feature_types_simple, ml_methods)
# Getting precision calculation error (divide by zero) on BRF for some folds - looks like it's labeling all 0 in some folds

In [ ]:
# Best method per feature set, based on mean AUC
best_results_df = results_df[results_df.groupby('feature_type')['AUC'].transform('max') == results_df['AUC']]
best_results_df

In [ ]:
best_methods = [best_results_df['method'].value_counts().idxmax()]
print(f'Method with best average AUC across EWS feature sets: {best_methods[0]}')

### 2: Train and evaluate best performing model on extended feature sets.

In [ ]:
main(data, feature_types_simple, best_methods, plot_curves=True)

In [ ]:
results_df_extended = main(data, feature_types_extended, best_methods)
results_df_extended

Comparing ML model with comparative Feature Sets for EWS:

Grabbing feature set with best performance for additional tests. 

Since Vital_Signs&Delta performs comparably to extended feature sets, might want to just use it for simplicity of readings/having fewer features relative to data points/ not overfit.

In [ ]:
best_feature_set = results_df_extended[results_df_extended['AUC'] == results_df_extended['AUC'].max()]['feature_type'].to_list()
print(f'Best feature set: {best_feature_set}')

### 3: Plot ROC curve for best featureset/model

In [ ]:
main(data, best_feature_set, best_methods, plot_curves=True)

### 4: Display most important features. TO IMPLEMENT AVG ACROSS CV

In [ ]:
#JA: issue: we are only selecting features from the final fold, and not plotting the other 4

# Top 20 features
#feats = pd.DataFrame()
#feats[['Top 20 features', 'Top feature_weight']] = fimp_df.head(20).reset_index(drop=True)
#feats[['Bottom 20 features', 'Bottom feature_weight']] = fimp_df.tail(20).reset_index(drop=True)

#top_20_features = list(fimp_df.head(20)['feature'])

#feats

In [ ]:
# Top 10 features
#feats_10 = pd.DataFrame()
#feats_10[['Top 10 features', 'Top feature_weight']] = fimp_df.head(10).reset_index(drop=True)
#feats_10[['Bottom 10 features', 'Bottom feature_weight']] = fimp_df.tail(10).reset_index(drop=True)

#feats_10

Performance on top 20 features only (mitigating overfitting)

In [ ]:
#results_df_threshold_opt = main(data, all_features_delta, best_methods, output, file_loc, top_n_features=top_20_features)
#results_df_threshold_opt

### 5: Additional feature sets

Adding age:

In [ ]:
main(data, ['All Features&Delta&age'], best_methods)

Additional experiment: 6hr and 12hr windows

In [ ]:
data_6hr = pd.read_csv(file_loc / 'data' / f"dataevents_updated2_N6_W24.csv")
data_6hr = data_manipulation(data_6hr)

main(data_6hr, best_feature_set, best_methods)

In [ ]:
data_12hr = = pd.read_csv(file_loc / 'data' / f"dataevents_updated2_N12_W24.csv")
data_12hr = data_manipulation(data_12hr)

main(data_12hr, best_feature_set, best_methods)

Performance including FiO2 as a variable.

In [ ]:
data_fio2 = pd.read_csv(file_loc / 'data' / f"dataevents_updated2_N24_W24.csv")
data_fio2 = data_manipulation(data_fio2, include_fio2=True)

main(data_fio2, best_feature_set, best_methods)